# Test

In [ ]:
import os
os.chdir('..')

import json
import tweepy
from modules import TwitterAuthentication
twitter = TwitterAuthentication()

## Metadata based on tweet ID

In [ ]:
# Set your API keys and access tokens
API_KEY = twitter.api_key
API_SECRET_KEY = twitter.api_key_secret
BEARER_TOKEN = twitter.bearer_token

# Authenticate with Tweepy
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET_KEY)

# Initialize Tweepy API client with the authentication handler
api = tweepy.Client(bearer_token=BEARER_TOKEN)

# Function to fetch a tweet with its expanded fields
def get_tweet_with_expanded_fields(tweet_id):
    tweet = api.get_tweet(
        tweet_id,
        tweet_fields=[
            "attachments", "author_id", "context_annotations", "created_at",
            "entities", "geo", "id", "in_reply_to_user_id", "lang",
            "possibly_sensitive", "public_metrics", "referenced_tweets",
            "source", "text", "withheld"
        ],
        expansions=["referenced_tweets.id"]
    )

    return tweet

# Example usage
tweet_id = "1212092628029698048"
tweet_data = get_tweet_with_expanded_fields(tweet_id)
print(tweet_data)


In [ ]:
# Extract data from the tweet
tweet_text = tweet_data.data.text
tweet_author_id = tweet_data.data.author_id
tweet_created_at = tweet_data.data.created_at
tweet_source = tweet_data.data.source

In [ ]:
print(tweet_text)

In [ ]:
print(tweet_created_at)

## Ingest tweets based on a specific query

In [ ]:
import requests

BEARER_TOKEN = twitter.bearer_token

def create_headers(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers

def search_tweets(headers, url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code}: {response.text}")
    return response.json()


In [ ]:
def create_url(query, max_results=10):
    url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&max_results={max_results}&tweet.fields=created_at,public_metrics,lang&expansions=author_id&user.fields=username"
    return url

def main():
    headers = create_headers(BEARER_TOKEN)
    query = "Sandra Torres"
    url = create_url(query)
    json_response = search_tweets(headers, url)

    for tweet in json_response["data"]:
        print(f"Tweet ID: {tweet['id']}")
        print(f"Tweet Text: {tweet['text']}")
        print(f"Tweet Created At: {tweet['created_at']}")
        print(f"Retweets: {tweet['public_metrics']['retweet_count']}")
        print(f"Likes: {tweet['public_metrics']['like_count']}")
        print(f"Replies: {tweet['public_metrics']['reply_count']}")
        print(f"Quote Counts: {tweet['public_metrics']['quote_count']}")
        print("------------")

In [ ]:
main()